# Summarizer Agent

In this notebook, we showcase how to create and use a Summarizer Agent that processes external web pages to produce concise summaries. The core idea involves:

1. **Filtering the Input**
   The agent checks for a web page URL in the user’s initial message. If a valid URL is present, it fetches the HTML content of that page.

2. **Preprocessing the Page**
   Before summarizing, the agent strips away HTML tags or other unnecessary data for efficient handling.

3. **Augmenting the Prompt**
   The agent then appends the cleaned-up page text to the user’s question, ensuring all necessary information is effectively included in the conversation.

4. **Producing a Short Summary**
   With the web page data in place, the agent can generate a concise summary of the page. If the user follows up with more questions about the same page, the agent can incorporate the previously retrieved content to continue the discussion.

By following this approach, the Summarizer Agent seamlessly combines downloading content with large language model capabilities, providing succinct answers that capture the essence of the requested web resource.

In [3]:
%useLatestDescriptors
@file:DependsOn("ai.ancf.lmos:arc-langchain4j-client:0.120.0")
@file:DependsOn("ai.ancf.lmos:arc-reader-html:0.120.0")
@file:DependsOn("dev.langchain4j:langchain4j-open-ai:1.0.0-beta1")

In [4]:
import ai.ancf.lmos.arc.agents.llm.ChatCompleter
import ai.ancf.lmos.arc.agents.llm.ChatCompletionSettings
import ai.ancf.lmos.arc.client.langchain4j.LangChainClient
import ai.ancf.lmos.arc.client.langchain4j.LangChainConfig
import dev.langchain4j.model.openai.OpenAiChatModel

val openAiApiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR-OPENAI-API-KEY"

val chatCompleterProvider: (String?) -> ChatCompleter = {
    LangChainClient(
        languageModel = LangChainConfig(
            modelName = "gpt-4",
            url = null,
            apiKey = openAiApiKey,
            accessKeyId = null,
            secretAccessKey = null,
        ), clientBuilder = { config, _ ->
            OpenAiChatModel.builder()
                .modelName(config.modelName)
                .apiKey(config.apiKey)
                .build()
        }
    )
}

In [5]:
import ai.ancf.lmos.arc.agents.DSLAgents
import ai.ancf.lmos.arc.agents.dsl.extensions.debug
import ai.ancf.lmos.arc.agents.dsl.extensions.extractUrl
import ai.ancf.lmos.arc.agents.dsl.extensions.html
import ai.ancf.lmos.arc.core.getOrThrow

val agentName = "summarizer-agent"
val agentBuilder = DSLAgents.init(chatCompleterProvider).apply {
    define {
        agent {
            name = agentName
            description = "Agent that summarizes web pages."
            prompt {
                """
                    You are a helpful agent.
                    You help customers by summarizing webpages.
                    Keep your answer short and concise.
                """
            }
            filterInput {
                val url = extractUrl(inputMessage).firstOrNull()
                if (url != null) {
                    debug("Loading url: $url")
                    val html = html(url).getOrThrow()
                    inputMessage = inputMessage.update(
                        """
                            User question: ${inputMessage.content}
                            The webpage $url contains the following text:
                            $html
                        """
                    )
                }
            }
        }
    }
}

In [6]:
val agents = agentBuilder.getAgents()
agents

[ChatAgent(name='summarizer-agent', description='Agent that summarizes web pages.')]

In [9]:
import ai.ancf.lmos.arc.agents.ChatAgent
import ai.ancf.lmos.arc.agents.User
import ai.ancf.lmos.arc.agents.conversation.Conversation
import ai.ancf.lmos.arc.agents.conversation.UserMessage
import ai.ancf.lmos.arc.agents.getAgentByName
import ai.ancf.lmos.arc.core.getOrNull
import kotlinx.coroutines.runBlocking

val articleUrl = "https://blog.jetbrains.com/kotlin/2025/02/kodees-kotlin-roundup-fresh-picks/"

val agent = agentBuilder.getAgentByName(agentName) as ChatAgent? ?: error("Agent not found!")
val conversation = Conversation(User("userOrClientId")) + UserMessage("Please summarize the following article: $articleUrl")
runBlocking {
    agent.execute(conversation).getOrNull()?.transcript?.getOrNull(1)?.content
}

The blog post on JetBrains titled "Kodee’s Kotlin Roundup: Fresh Picks to Begin 2025" discusses recent developments and updates in the Kotlin language. Highlights include the release of Kotlin 2.1.0 and the subsequent arrival of Kotlin 2.1.20-Beta2, which brings in tooling updates, new language features, and performance improvements. The blog also announces the introduction of klibs.io for easier discovery of Kotlin Multiplatform libraries and the winners of the Kotlin Multiplatform Contest. Additionally, it provides information about Kotlin's presence in the Google Summer of Code 2024 and a program for server-side Kotlin content creators. Finally, it mentions upcoming events, including KotlinConf 2025 and Kotlin workshops on May 21.
